In [1]:
%reload_ext autoreload
%autoreload 2

import sys
import pandas as pd
sys.path.insert(0, './Moduulit')
pd.options.mode.chained_assignment = None

# Importtaus tässä muodossa
from database_connection import database_query
from outliers import find_outliers, draw_histogram
from clean_dataframe import clean_dataframe
from draw_node_amount import *
from write_df_to_database import write_df_to_mariadb
from xy_diskretisointi import xy_to_grid
from isolation_forest import *
from reittien_erottelu import *
from sisään_uloskäynnit import *
from nopeudet import *
from LOF import *
from clean_velocities import *
from draw_route_grid import *

### Read data from database
###### 

In [ ]:
%%time
df = database_query("SELECT * FROM SensorData LIMIT 100000000")
alussa = len(df)
df

### Find and delete outliers
###### https://gitlab.dclabra.fi/ryhm-fox/projekti-2-team-fox/-/blob/master/Dokumentaatio/Datan%20Siivous/Z-score%20Outlierit.md


In [ ]:
%%time
df = find_outliers(df)
outlierit = len(df)

### Edit timestamps in right form, add hour and weekday columns, and remove useless columns
###### https://gitlab.dclabra.fi/ryhm-fox/projekti-2-team-fox/-/blob/master/Dokumentaatio/Datan%20Siivous/clean_dataframe.md


In [ ]:
%%time
df = clean_dataframe(df)
aukiolot = len(df)

# LocalOutlierFactor

In [ ]:
%%time
df2, outlierit = LOF(df, 10, 10)
# Draw the deleted and kept points
draw_LOF(df2, outlierit)
# Do some stuff to calculate the amount dropped.
df3 = df[~df.isin(df2)].dropna()
df1 = df3.copy()
df_len_LOF = len(df1)

In [31]:
df1 = df3.copy()
df1

,node_id,timestamp,x,y,dayofweek,current_hour
28,1,2020-06-24 15:20:56.088,3560.0,2009.0,2,8
29,1,2020-06-24 15:20:59.798,3559.0,2250.0,2,8
30,1,2020-06-24 15:21:11.232,3152.0,2963.0,2,8
31,1,2020-06-24 15:21:12.231,3013.0,2793.0,2,8
32,1,2020-06-24 15:21:13.512,2923.0,2740.0,2,8
...,...,...,...,...,...,...
9725298,31,2020-09-30 12:24:13.062,2711.0,-311.0,2,5
9725299,31,2020-09-30 12:24:50.062,2718.0,-318.0,2,5
9725300,31,2020-09-30 12:25:36.062,2721.0,-324.0,2,5
9725301,31,2020-09-30 12:26:03.062,2719.0,-323.0,2,5


### Calculate and dd distance column

In [5]:
%%time
df = nopeudet(df)
nopeus = len(df)

CPU times: user 1.78 s, sys: 720 ms, total: 2.5 s
Wall time: 2.5 s


In [ ]:
%%time
df2 = velocity.clean_vel(df1, 'x', 'y')
nopeus = len(df2)
draw_vel(df1,df2, 'x', 'y')
df2

In [ ]:
df = df2

### Set distance to match gridsize

In [ ]:
%%time
grid_size = 40
x_cut = pd.cut(df['distance'], grid_size,labels=False)
# Laitetaan saadut arvot dataframeen
df['distance'] = x_cut.values

### Discretize x- and y-coordinates, and get grid_id based on discretized coordinates.
###### https://gitlab.dclabra.fi/ryhm-fox/projekti-2-team-fox/-/blob/master/Dokumentaatio/Grid/XY_Diskretisointi.md

In [ ]:
df = xy_to_grid(df, df.x, df.y, grid_size)
df

### Get entry- and cashier area coordinates. Get grid_ids based on those coordinates.
###### https://gitlab.dclabra.fi/ryhm-fox/projekti-2-team-fox/-/blob/master/Dokumentaatio/Data-analyysi/Sis%C3%A4%C3%A4n_ja_ulosk%C3%A4ynnit.md

In [ ]:
in_x, in_y = sisään(df, 'x_grid', 'y_grid')
out_x, out_y = ulos(df, 'x_grid', 'y_grid')

draw_exits(df, in_x, in_y, out_x, out_y, "x_grid", "y_grid")

# Muokataan sisäänkäynnin ja kassojen sijainnit ID-muotoon.
in_ID = xy_to_ID(in_x,in_y, grid_size)
out_ID = xy_to_ID(out_x,out_y, grid_size)

# Haetaan uniikit id:t
u_in_ID = in_ID.unique()
u_out_ID = out_ID.unique()


# # Voidaan tarkistaa sijaintien oikeellisuus
# sns.scatterplot(x="x_grid", y="y_grid", data = df)
# sns.scatterplot(x=in_x, y=in_y)
# sns.scatterplot(x=out_x, y=out_y)
# hmax= sns.kdeplot(df.x_grid, df.y_grid, cmap="afmhot", shade=True, alpha = 0.7)
# hmax.collections[3].set_alpha(0)
# plt.show()

### Separate shopping routes from each other
###### https://gitlab.dclabra.fi/ryhm-fox/projekti-2-team-fox/-/blob/master/Dokumentaatio/Reitit/Reittien_erottelu.md

In [ ]:
reitit = erottele_reitit(df, u_in_ID, u_out_ID)
reitit = poista_lyhyet_reitit(reitit, grid_size)
df2 = reitit_dataframeksi(reitit)
erottelu=len(df2)

### Draw routes

In [ ]:
draw_route_grid(df2, 'ajokerta')

### Remove datapoints from cashier- and entry area. Remove those shopping trips, which were shopping longer than 2 hours. 

In [ ]:
# poistetaan kassa- ja sisääntuloalueella olevat datapisteet
df = df[~df["grid_id"].isin(u_in_ID)]
df = df[~df["grid_id"].isin(u_out_ID)]

alueet = len(df)

# poistetaan reitit, joiden kesto ei ole järkevä
df["kesto"] = df["kesto"].dt.total_seconds()
df = df[df["kesto"] < 7200]
df = df[df["kesto"] > 0]

kestot = len(df)

pd.DataFrame.from_dict([{"Alussa":alussa, "Z-Score":outlierit, "Aukioloajat":aukiolot,"Nopeus":nopeus, "Reittien erottelu":erottelu, "Kauppa-alue":alueet, "Väärät kestot":kestot}]).T.plot.bar(figsize=(12,8), rot=360, ylabel="milj.", title="Datan määrä eri operaatioiden jälkeen.")
df

In [11]:
df

,ajokerta,node_id,timestamp,x,y,grid_id,distance,kesto,dayofweek,current_hour
9,2,1,2020-06-25 15:21:46.646,28,15,628,0,6812.007,3,8
10,2,1,2020-06-25 15:21:47.645,28,15,628,0,6812.007,3,8
11,2,1,2020-06-25 15:21:47.645,28,15,628,0,6812.007,3,8
12,2,1,2020-06-25 15:21:47.647,28,15,628,0,6812.007,3,8
13,2,1,2020-06-25 15:21:50.645,26,15,626,1,6812.007,3,8
...,...,...,...,...,...,...,...,...,...,...
72,9442,31,2020-09-22 14:41:04.037,19,16,659,0,97.531,1,7
73,9442,31,2020-09-22 14:41:05.038,19,16,659,0,97.531,1,7
75,9442,31,2020-09-22 14:41:06.037,18,16,658,0,97.531,1,7
84,9442,31,2020-09-22 14:41:11.570,1,28,1121,21,97.531,1,7


### Save cleaned shopping trips to database
###### https://gitlab.dclabra.fi/ryhm-fox/projekti-2-team-fox/-/blob/master/Dokumentaatio/write_df_to_dataframe.md

In [ ]:
for i in df.ajokerta.unique():
    print(df[df["ajokerta"] == i].distance.sum()/ df[df["ajokerta"] == i].kesto.mean())
# df
df.ajokerta.unique()

In [12]:
write_df_to_mariadb(df, "REITIT")

Existing table found. Prepairing to Drop Table named REITIT...


Continue? yes/no:  yes


Done! Prepairing to write dataframe to REITIT
Writing data 1 / 14
Writing data 2 / 14
Writing data 3 / 14
Writing data 4 / 14
Writing data 5 / 14
Writing data 6 / 14
Writing data 7 / 14
Writing data 8 / 14
Writing data 9 / 14
Writing data 10 / 14
Writing data 11 / 14
Writing data 12 / 14
Writing data 13 / 14
Writing data 14 / 14
Done!
